In [1]:
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
sys.path.append('../')
import utils.lib as lib
from easydict import EasyDict
from config.config import config
from utils.phase import train
from utils.timeseriesv1 import TimeSeriesDataset
from utils.metrics import mape_and_compliance_rate

config = EasyDict(config)
data_config = config['data_config']
config.model_dir = '../ckpt/qishoufu-online'

pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows', None)
# np.set_printoptions(threshold=np.nan)

df = pd.read_pickle('/Users/mtdp/workspace/timesereis-dataset/processed_new/qishoufu-online.pkl')
for i, v in enumerate(df.columns.to_list()):
    print(i, v)
    
for i, v in enumerate(df['label']):
    print(i, v)
    
class BusinessLine(TimeSeriesDataset):
    def __init__(self, para, df):
        self.df = df
        input_size = self.df.values.shape[1]
        self.scale = np.zeros(input_size, np.float32)
        self.mn = np.zeros(input_size, np.float32)
        dat = np.zeros(self.df.values.shape, np.float32)
        for i in range(input_size):
            self.mn[i] = np.min(self.df.values[:, i])
            self.scale[i] = np.max(self.df.values[:, i]) - self.mn[i]
            dat[:, i] = (self.df.values[:, i] - self.mn[i]) / self.scale[i]
            
        norm_df = pd.DataFrame(dat, columns=self.df.columns)
        super(BusinessLine, self).__init__(para, norm_df)

num_columns = df.values.shape[1]
df1 = df.copy()
df1.columns = [str(i) for i in range(df.values.shape[1])]
df1 = df1[[str(i) for i in range(1, num_columns)]]

data_config.x.key = [str(i) for i in range(1, num_columns)]
data_config.y.key = ['1']
data_config.z.key = [str(i) for i in range(2, 49)]
# data_config.shuffle = True
data_config.normalization = "global"
df1 = df1.iloc[:905]
df1

0 date
1 label
2 order
3 day_month
4 rest_flag
5 holiday_flag
6 feat_month1
7 feat_month2
8 feat_month3
9 feat_month4
10 feat_month5
11 feat_month6
12 feat_month7
13 feat_month8
14 feat_month9
15 feat_month10
16 feat_month11
17 feat_month12
18 feat_day0
19 feat_day1
20 feat_day2
21 feat_day3
22 feat_day4
23 feat_day5
24 feat_day6
25 feat_holiday0
26 feat_holiday1
27 feat_holiday2
28 feat_holiday3
29 feat_holiday4
30 feat_holiday5
31 feat_holiday6
32 feat_holiday7
33 feat_holiday_aux0
34 feat_holiday_aux1
35 feat_holiday_aux2
36 feat_holiday_aux3
37 feat_holiday_aux4
38 feat_holiday_aux5
39 feat_holiday_aux6
40 feat_holiday_aux7
41 feat_holiday_aux8
42 feat_holiday_aux9
43 feat_holiday_aux10
44 feat_holiday_aux11
45 feat_holiday_aux12
46 feat_holiday_aux13
47 feat_holiday_aux14
48 feat_holiday_aux15
49 ('weather_from', 'beijing')
50 ('weather_from', 'chengdu')
51 ('weather_from', 'chongqing')
52 ('weather_from', 'guangzhou')
53 ('weather_from', 'hangzhou')
54 ('weather_from', 'nanjing')

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114
0,171,8.07184e+06,0.193548,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.2,0.1,0.2,0.1,0.0,0.1,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.2,0.1,0.1,0.2,0.1,0.1,0.2,0.1,0.1,0.200000,0.466667,0.400000,0.766667,0.566667,0.500000,0.533333,0.733333,0.166667,0.466667,0.266667,-0.100000,0.266667,0.366667,0.433333,0.233333,0.133333,0.300000,0.566667,-0.033333,0.133333,0.000000,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,176,8.18388e+06,0.000000,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.2,0.1,0.1,0.1,0.1,0.0,0.0,0.0,0.2,0.2,0.1,0.2,0.1,0.1,0.1,0.1,0.0,0.0,0.1,0.2,0.200000,0.466667,0.433333,0.833333,0.533333,0.466667,0.500000,0.800000,0.133333,0.533333,0.266667,-0.200000,0.066667,0.333333,0.500000,0.266667,0.133333,0.266667,0.600000,-0.133333,0.133333,0.066667,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,227,7.46113e+06,0.032258,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.2,0.2,0.2,0.0,0.1,0.1,0.0,0.1,0.2,0.1,0.2,0.2,0.2,0.2,0.1,0.1,0.2,0.1,0.1,0.2,0.2,0.2,0.166667,0.400000,0.466667,0.866667,0.533333,0.433333,0.466667,0.866667,0.166667,0.500000,0.266667,-0.166667,0.200000,0.266667,0.533333,0.266667,0.166667,0.266667,0.633333,-0.100000,0.133333,0.000000,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,211,7.73235e+06,0.064516,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.2,0.2,0.2,0.0,0.3,0.3,0.3,0.1,0.2,0.3,0.2,0.2,0.2,0.2,0.1,0.3,0.4,0.4,0.1,0.1,0.4,0.2,0.200000,0.400000,0.466667,0.900000,0.500000,0.400000,0.500000,0.800000,0.033333,0.433333,0.333333,-0.166667,0.233333,0.366667,0.566667,0.366667,0.300000,0.366667,0.600000,-0.066667,0.333333,0.133333,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,205,8.1605e+06,0.096774,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.4,0.0,0.2,0.1,0.4,0.4,0.4,0.1,0.2,0.4,0.3,0.2,0.1,0.2,0.1,0.3,0.3,0.3,0.1,0.2,0.4,0.3,0.066667,0.466667,0.466667,0.833333,0.433333,0.366667,0.400000,0.866667,0.100000,0.333333,0.200000,-0.133333,0.166667,0.366667,0.566667,0.366667,0.266667,0.333333,0.633333,-0.033333,0.233333,0.133333,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.3,0.1,0.1,0.1,0.1,0.1
5,206,7.99793e+06,0.129032,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.2,0.2,0.2,0.1,0.3,0.3,0.3,0.1,0.1,0.3,0.3,0.2,0.2,0.2,0.2,0.4,0.4,0.4,0.1,0.2,0.3,0.3,0.133333,0.400000,0.466667,0.866667,0.400000,0.366667,0.466667,0.833333,0.133333,0.266667,0.166667,-0.066667,0.200000,0.333333,0.566667,0.333333,0.233333,0.366667,0.633333,0.000000,0.200000,0.066667,0.1,0.1,0.1,0.1,0.1,0.3,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.3,0.1,0.1,0.1,0.1,0.1
6,171,8.80628e+06,0.161290,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.4,0.1,0.2,0.3,0.4,0.3,0.3,0.1,0.4,0.3,0.2,0.2,0.3,0.1,0.3,0.3,0.2,0.2,0.1,0.4,0.0,0.1,0.100000,0.400000,0.466667,0.833333,0.366667,0.366667,0.466667,0.866667,0.133333,0.266667,0.200000,-0.100000,0.200000,0.300000,0.600000,0.300000,0.200000,0.266667,0.566667,-0.033333,0.066667,-0.033333,0.1,0.1,1.0,0.1,0.1,0.3,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.3,0.1,0.1,0.1,0.1,0.1
7,203,8.38733e+06,0.193548,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.2,0.2,0.1,0.1,0.2,0.3,0.1,0.0,0.0,0.1,0.0,0.3,0.2,0.1,0.1,0.1,0.2,0.1,0.0,0.1,

In [2]:
mts = BusinessLine(data_config, df1)

number of sample is 842...


In [3]:
VERSION = "tpa_bidirectional_lstm"

train_graph, train_model = lib.create_graph(config, 'train', version=VERSION) 
with tf.Session(config=lib.config_setup(), graph=train_graph) as train_sess:
    lib.train_load_weight(config.model_dir, train_sess, train_model)
    valid_graph, valid_model = lib.create_graph(config, 'validation', version=VERSION)
    valid_sess = tf.Session(config=lib.config_setup(), graph=valid_graph)
    min_mape = 1
    final_compliance_rate = 0
    for i in range(config.num_epoch):
        while not mts.train_status['stop']:
            x, y, z = mts.get_train_batch()
            loss, global_step = None, None
            if VERSION in ['single_tpa']:
                loss, global_step, _ = train_sess.run([train_model.loss, train_model.global_step, train_model.update], 
                                                      feed_dict={train_model.x: x, 
                                                                 train_model.y: y})
                
            else:
                loss, global_step, _ = train_sess.run([train_model.loss, train_model.global_step, train_model.update], 
                                                      feed_dict={train_model.x: x, 
                                                                 train_model.y: y, 
                                                                 train_model.z: z})
            if global_step % 1 == 0:
                print('global_step:{} loss:{}'.format(global_step, loss))
        mts.train_initializer()
        [global_step] = train_sess.run([train_model.global_step])
        train_model.saver.save(train_sess, os.path.join(config.model_dir, 'model.ckpt'), global_step=global_step)
        
        
        lib.eval_load_weight(config.model_dir, valid_sess, valid_model)

        all_outputs = []
        all_labels = []
        valid_loss = 0
        while not mts.validation_status['stop']:
            x, y, z = mts.get_validation_batch()
            loss, outputs, labels = None, None, None
            if VERSION in ['single_tpa']:
                loss, outputs, labels = valid_sess.run([valid_model.loss, valid_model.future_tpa_outputs, valid_model.y], 
                                                       feed_dict={valid_model.x: x, 
                                                                  valid_model.y: y})
                
            else:
                loss, outputs, labels = valid_sess.run([valid_model.loss, valid_model.final_outputs, valid_model.y], 
                                                       feed_dict={valid_model.x: x, 
                                                                  valid_model.y: y, 
                                                                  valid_model.z: z})
                
            all_outputs.append(outputs)
            all_labels.append(labels)
            valid_loss += loss

        mts.validation_initializer()
        all_outputs = np.concatenate(all_outputs, axis=0)
        all_labels = np.concatenate(all_labels, axis=0)

        valid_mape, compliance_rate = mape_and_compliance_rate(all_outputs, all_labels)
        if valid_mape < min_mape:
            min_mape = valid_mape
            final_compliance_rate = compliance_rate
            valid_model.saver.save(valid_sess, os.path.join(config.model_dir + '/best', 'model.ckpt'), global_step=global_step)

        print('MAPE:{} Compliance_Rate:{} MIN_MAPE:{} MIN_Compliance_Rate: {} loss:{}'.format(valid_mape, 
                                                                                              compliance_rate, 
                                                                                              min_mape, 
                                                                                              final_compliance_rate, 
                                                                                              valid_loss))
    config['batch_size'] = 1   
    test_graph, test_model = lib.create_graph(config, 'test', VERSION)
    test_sess =  tf.Session(config=lib.config_setup(), graph=test_graph)                                                                               
    lib.eval_load_weight(config.model_dir, test_sess, test_model)
    
    all_outputs = []
    all_labels = []
    valid_loss = 0
    while not mts.test_status['stop']:
        x, y, z = mts.get_test_batch()
        if VERSION in ['single_tpa']:
            loss, outputs, labels = test_sess.run([test_model.loss, test_model.future_tpa_outputs, test_model.y], 
                                                   feed_dict={test_model.x: x, 
                                                              test_model.y: y})
                
        else:
            loss, outputs, labels = test_sess.run([test_model.loss, test_model.final_outputs, test_model.y], 
                                                   feed_dict={test_model.x: x, 
                                                              test_model.y: y, 
                                                              test_model.z: z})
                
        all_outputs.append(outputs)
        all_labels.append(labels)
        valid_loss += loss

    all_outputs = np.concatenate(all_outputs, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    test_mape, compliance_rate = mape_and_compliance_rate(all_outputs, all_labels)
    print("Testing...")
    print('MAPE:{} compliance_rate:{}'.format(test_mape, compliance_rate))
        
        
        

Building train graph...
Building loss
Building optimizer...
Building eval graph...
Building loss
global_step:1 loss:0.18869397044181824
global_step:2 loss:0.7358711361885071
global_step:3 loss:0.29225876927375793
global_step:4 loss:0.6273250579833984
global_step:5 loss:0.6113004684448242
global_step:6 loss:0.34004101157188416
global_step:7 loss:0.33059248328208923
global_step:8 loss:0.28061869740486145
global_step:9 loss:0.1776312291622162
global_step:10 loss:0.1478005349636078
global_step:11 loss:0.15094587206840515
global_step:12 loss:0.1229722872376442
global_step:13 loss:0.11132816225290298
global_step:14 loss:0.12606868147850037
global_step:15 loss:0.10093820840120316
global_step:16 loss:0.09083711355924606
global_step:17 loss:0.11311434954404831
global_step:18 loss:0.09306498616933823
global_step:19 loss:0.10127312690019608
global_step:20 loss:0.08058340102434158
global_step:21 loss:0.10154969245195389
global_step:22 loss:0.09196930378675461
global_step:23 loss:0.0922726392745971

global_step:173 loss:0.014832928776741028
global_step:174 loss:0.020270254462957382
global_step:175 loss:0.01697564125061035
global_step:176 loss:0.013344856910407543
global_step:177 loss:0.014697598293423653
global_step:178 loss:0.020421182736754417
global_step:179 loss:0.015119628980755806
global_step:180 loss:0.020967381075024605
global_step:181 loss:0.014990204013884068
global_step:182 loss:0.016454173251986504
global_step:183 loss:0.019516978412866592
global_step:184 loss:0.013361239805817604
global_step:185 loss:0.031922779977321625
global_step:186 loss:0.01717652939260006
global_step:187 loss:0.021329905837774277
global_step:188 loss:0.02195444330573082
global_step:189 loss:0.012492232024669647
global_step:190 loss:0.01825157180428505
global_step:191 loss:0.02221617102622986
global_step:192 loss:0.017026236280798912
INFO:tensorflow:Restoring parameters from ../ckpt/qishoufu-online/model.ckpt-192
MAPE:0.2860856354236603 Compliance_Rate:0.0 MIN_MAPE:0.2747526168823242 MIN_Complian

global_step:342 loss:0.012179906480014324
global_step:343 loss:0.01168315764516592
global_step:344 loss:0.012438258156180382
global_step:345 loss:0.0125108128413558
global_step:346 loss:0.012870856560766697
global_step:347 loss:0.014595657587051392
global_step:348 loss:0.014038397930562496
global_step:349 loss:0.013252434320747852
global_step:350 loss:0.01605965383350849
global_step:351 loss:0.012224380858242512
global_step:352 loss:0.010674732737243176
INFO:tensorflow:Restoring parameters from ../ckpt/qishoufu-online/model.ckpt-352
MAPE:0.2895979583263397 Compliance_Rate:0.02976190476190476 MIN_MAPE:0.1927863508462906 MIN_Compliance_Rate: 0.20535714285714285 loss:0.31372928246855736
global_step:353 loss:0.012990881688892841
global_step:354 loss:0.010689596645534039
global_step:355 loss:0.012424483895301819
global_step:356 loss:0.012824575416743755
global_step:357 loss:0.012303384952247143
global_step:358 loss:0.010377396829426289
global_step:359 loss:0.010331787168979645
global_step:3

global_step:511 loss:0.012767762877047062
global_step:512 loss:0.02234174869954586
INFO:tensorflow:Restoring parameters from ../ckpt/qishoufu-online/model.ckpt-512
MAPE:0.2617841362953186 Compliance_Rate:0.23511904761904762 MIN_MAPE:0.1927863508462906 MIN_Compliance_Rate: 0.20535714285714285 loss:0.28505638986825943
global_step:513 loss:0.014730741269886494
global_step:514 loss:0.01879115216434002
global_step:515 loss:0.01864076592028141
global_step:516 loss:0.012790242210030556
global_step:517 loss:0.029823848977684975
global_step:518 loss:0.016587255522608757
global_step:519 loss:0.012543496675789356
global_step:520 loss:0.014461168088018894
global_step:521 loss:0.010818367823958397
global_step:522 loss:0.011400952935218811
global_step:523 loss:0.012361160479485989
global_step:524 loss:0.010923166759312153
global_step:525 loss:0.013059958815574646
global_step:526 loss:0.011375928297638893
global_step:527 loss:0.014429130591452122
global_step:528 loss:0.012026645243167877
global_step:

global_step:674 loss:0.008956014178693295
global_step:675 loss:0.009670295752584934
global_step:676 loss:0.00832082238048315
global_step:677 loss:0.007381357718259096
global_step:678 loss:0.00751247676089406
global_step:679 loss:0.007882915437221527
global_step:680 loss:0.00848890095949173
global_step:681 loss:0.01119869016110897
global_step:682 loss:0.009665333665907383
global_step:683 loss:0.012950737960636616
global_step:684 loss:0.016720479354262352
global_step:685 loss:0.010437145829200745
global_step:686 loss:0.015619601123034954
global_step:687 loss:0.01130203902721405
global_step:688 loss:0.015742404386401176
global_step:689 loss:0.014870350249111652
global_step:690 loss:0.011271698400378227
global_step:691 loss:0.01718086190521717
global_step:692 loss:0.012716412544250488
global_step:693 loss:0.016156528145074844
global_step:694 loss:0.01711350865662098
global_step:695 loss:0.012892688624560833
global_step:696 loss:0.013455367647111416
global_step:697 loss:0.01952407881617546


...

global_step:1825 loss:0.006697326432913542
global_step:1826 loss:0.010357116349041462
global_step:1827 loss:0.008092088624835014
global_step:1828 loss:0.007839570753276348
global_step:1829 loss:0.007233395706862211
global_step:1830 loss:0.006084308959543705
global_step:1831 loss:0.006600373424589634
global_step:1832 loss:0.006590318400412798
global_step:1833 loss:0.006398883182555437
global_step:1834 loss:0.008293968625366688
global_step:1835 loss:0.004821414593607187
global_step:1836 loss:0.005750205367803574
global_step:1837 loss:0.006395355798304081
global_step:1838 loss:0.006463457364588976
global_step:1839 loss:0.008423158898949623
global_step:1840 loss:0.006079989019781351
global_step:1841 loss:0.004914365243166685
global_step:1842 loss:0.0054611810483038425
global_step:1843 loss:0.006455318536609411
global_step:1844 loss:0.007287905551493168
global_step:1845 loss:0.006888287607580423
global_step:1846 loss:0.00834393035620451
global_step:1847 loss:0.006815016269683838
global_step

global_step:1989 loss:0.0060580153949558735
global_step:1990 loss:0.005228803027421236
global_step:1991 loss:0.006991864647716284
global_step:1992 loss:0.006815627217292786
global_step:1993 loss:0.007760148961097002
global_step:1994 loss:0.007639558054506779
global_step:1995 loss:0.009340367279946804
global_step:1996 loss:0.009475436992943287
global_step:1997 loss:0.008305837400257587
global_step:1998 loss:0.011280683800578117
global_step:1999 loss:0.009301082231104374
global_step:2000 loss:0.008499023504555225
global_step:2001 loss:0.010180866345763206
global_step:2002 loss:0.008110671304166317
global_step:2003 loss:0.007407783064991236
global_step:2004 loss:0.010875369422137737
global_step:2005 loss:0.00928395427763462
global_step:2006 loss:0.008874624967575073
global_step:2007 loss:0.00856618583202362
global_step:2008 loss:0.00734300259500742
global_step:2009 loss:0.007808498572558165
global_step:2010 loss:0.008617214858531952
global_step:2011 loss:0.0054647973738610744
global_step:

global_step:2153 loss:0.006596401333808899
global_step:2154 loss:0.005338757298886776
global_step:2155 loss:0.006912884768098593
global_step:2156 loss:0.006552503444254398
global_step:2157 loss:0.005972946994006634
global_step:2158 loss:0.006155109964311123
global_step:2159 loss:0.008218003436923027
global_step:2160 loss:0.005635249428451061
global_step:2161 loss:0.00855548121035099
global_step:2162 loss:0.006567018572241068
global_step:2163 loss:0.005839947611093521
global_step:2164 loss:0.00901526678353548
global_step:2165 loss:0.007769566494971514
global_step:2166 loss:0.007319082506000996
global_step:2167 loss:0.009644584730267525
global_step:2168 loss:0.005656111519783735
global_step:2169 loss:0.007816724479198456
global_step:2170 loss:0.009038853459060192
global_step:2171 loss:0.007389035541564226
global_step:2172 loss:0.007457766216248274
global_step:2173 loss:0.008081749081611633
global_step:2174 loss:0.005739438813179731
global_step:2175 loss:0.007068771403282881
global_step:2

global_step:2316 loss:0.005477363709360361
global_step:2317 loss:0.007502367254346609
global_step:2318 loss:0.007065088488161564
global_step:2319 loss:0.006795028690248728
global_step:2320 loss:0.006597604602575302
global_step:2321 loss:0.009252761490643024
global_step:2322 loss:0.005491062067449093
global_step:2323 loss:0.006917589344084263
global_step:2324 loss:0.006427524145692587
global_step:2325 loss:0.008897610940039158
global_step:2326 loss:0.006883802358061075
global_step:2327 loss:0.006116063334047794
global_step:2328 loss:0.007030159700661898
global_step:2329 loss:0.006044060457497835
global_step:2330 loss:0.005907643120735884
global_step:2331 loss:0.004807437304407358
global_step:2332 loss:0.006489173509180546
global_step:2333 loss:0.005999863147735596
global_step:2334 loss:0.007508598268032074
global_step:2335 loss:0.006530596408993006
global_step:2336 loss:0.0063964128494262695
INFO:tensorflow:Restoring parameters from ../ckpt/qishoufu-online/model.ckpt-2336
MAPE:0.2440579

global_step:2480 loss:0.006947587709873915
global_step:2481 loss:0.006273824255913496
global_step:2482 loss:0.007405516225844622
global_step:2483 loss:0.005512402392923832
global_step:2484 loss:0.0054990011267364025
global_step:2485 loss:0.006583082489669323
global_step:2486 loss:0.006338019389659166
global_step:2487 loss:0.00659154774621129
global_step:2488 loss:0.005935611668974161
global_step:2489 loss:0.005494024604558945
global_step:2490 loss:0.005791319999843836
global_step:2491 loss:0.0046852752566337585
global_step:2492 loss:0.0058423676528036594
global_step:2493 loss:0.004411786794662476
global_step:2494 loss:0.004511810373514891
global_step:2495 loss:0.004701224621385336
global_step:2496 loss:0.005469128023833036
INFO:tensorflow:Restoring parameters from ../ckpt/qishoufu-online/model.ckpt-2496
MAPE:0.25141701102256775 Compliance_Rate:0.3630952380952381 MIN_MAPE:0.1927863508462906 MIN_Compliance_Rate: 0.20535714285714285 loss:0.27467399649322033
global_step:2497 loss:0.0056145